## TFRecord 파일 검사
- TFRecord의 구조와 포함된 데이터 유형을 확인

In [6]:
import tensorflow as tf

def inspect_tfrecord(tfrecord_path):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print(example)

# 사용 예시
tfrecord_path = 'your_tfrecord_file_path' # 이 부분을 실제 경로로 교체 필요
inspect_tfrecord(tfrecord_path)


features {
  feature {
    key: "image/width"
    value {
      int64_list {
        value: 416
      }
    }
  }
  feature {
    key: "image/object/class/text"
    value {
      bytes_list {
        value: "weeds"
      }
    }
  }
  feature {
    key: "image/object/class/label"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "image/object/bbox/ymin"
    value {
      float_list {
        value: 0.735576928
      }
    }
  }
  feature {
    key: "image/object/bbox/ymax"
    value {
      float_list {
        value: 0.788461566
      }
    }
  }
  feature {
    key: "image/object/bbox/xmin"
    value {
      float_list {
        value: 0.725961566
      }
    }
  }
  feature {
    key: "image/object/bbox/xmax"
    value {
      float_list {
        value: 0.783653855
      }
    }
  }
  feature {
    key: "image/height"
    value {
      int64_list {
        value: 416
      }
    }
  }
  feature {
    key: "image/format"
    value {
      bytes_l

## 이미지 데이터 추출 및 저장
- TFRecord에서 이미지 데이터를 추출하고 저장하는 과정을 자동화

In [9]:
import tensorflow as tf
from PIL import Image
import numpy as np

def parse_tfrecord(example_proto):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string, default_value='jpeg'),
    }
    return tf.io.parse_single_example(example_proto, feature_description)

def convert_tfrecord_to_images(tfrecord_path, output_dir):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    parsed_dataset = raw_dataset.map(parse_tfrecord)
    
    for i, record in enumerate(parsed_dataset):
        # 이미지 디코딩
        image = tf.io.decode_jpeg(record['image/encoded'])
        
        # numpy 배열로 변환
        image_np = image.numpy()
        
        # 이미지 저장
        img = Image.fromarray(image_np)
        img.save(f"{output_dir}/image_{i}.jpg")

# 사용 예시
tfrecord_path = 'your_tfrecord_file_path' # 실제 경로로 교체 필요
output_dir =  'your_output_directory_path' # 실제 출력 디렉토리 경로로 교체 필요
convert_tfrecord_to_images(tfrecord_path, output_dir)


## 이미지 다운스케일링 및 저장

이 노트북에서는 High Resolution (HR) 이미지를 Low Resolution (LR) 이미지로 다운스케일링하고, 이를 저장하는 과정을 다룹니다. PIL 라이브러리를 사용하여 이미지 크기를 조정하며, 이를 통해 Super Resolution (SR) 모델 훈련에 필요한 데이터를 준비할 수 있습니다.

### 주요 단계
1. **이미지 파일 로드**: 지정된 경로에서 HR 이미지를 로드합니다.
2. **이미지 다운스케일링**: 주어진 스케일 팩터를 사용하여 HR 이미지를 LR 이미지로 변환합니다.
3. **LR 이미지 저장**: 변환된 LR 이미지를 지정된 디렉토리에 저장합니다.
4. **디렉토리 내 모든 이미지 처리**: HR 이미지가 포함된 디렉토리 내의 모든 이미지를 처리하고, LR 이미지로 변환하여 다른 디렉토리에 저장합니다.


In [12]:
import os
from PIL import Image

def downscale_and_save_image(image_path, scale_factor=4, lr_output_path=None):
    # 이미지 로드
    image = Image.open(image_path)
    
    # 원본 HR 이미지 크기
    hr_width, hr_height = image.size
    
    # LR 이미지 크기 계산
    lr_width, lr_height = hr_width // scale_factor, hr_height // scale_factor
    
    # 이미지 다운스케일링
    lr_image = image.resize((lr_width, lr_height), Image.BICUBIC)
    
    # 이미지 파일 이름 분리
    file_name = os.path.basename(image_path)
    
    # LR 이미지 저장
    if lr_output_path:
        lr_image.save(os.path.join(lr_output_path, file_name))
    
    return lr_image

def process_directory(hr_directory, lr_directory, scale_factor=4):
    # 디렉토리 내의 모든 파일 검색
    for file_name in os.listdir(hr_directory):
        # 파일 경로 생성
        file_path = os.path.join(hr_directory, file_name)
        
        # 이미지 파일인지 확인
        if os.path.isfile(file_path) and file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            # 이미지 다운스케일링 및 저장
            downscale_and_save_image(file_path, scale_factor, lr_directory)

# 사용 예시
hr_directory = 'your_hr_directory_path'  # 실제 HR 이미지 디렉토리 경로로 교체하세요
lr_directory = 'your_lr_directory_path'  # 실제 LR 이미지 디렉토리 경로로 교체하세요
process_directory(hr_directory, lr_directory, scale_factor=4)
